In [1]:
import pandas as pd
import json
import string

In [16]:
all_files_za = pd.read_csv("Classification/all_files_ZAv1.csv")

In [17]:
all_files_za.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time'],
      dtype='object')

In [18]:
all_files_za = all_files_za[["End Date", "artist_names", "main_artist", "track_name", "streams", 'source', "Year"]]

In [26]:
def groupbysum(all_files, year, relevantcols, grouplist, sumcol, sortcol, isascending, isYear, filepath):
    if(isYear):
        za2022 = all_files[all_files["Year"] == year]
    else:
        za2022 = all_files

#     print(za2022.shape)

    za2022df = za2022[relevantcols].groupby(grouplist)[sumcol].apply(sum)

    za2022dsummed = za2022df.reset_index()

    zasummedsorted = za2022dsummed.sort_values(by=sortcol, ascending=isascending)

    zasummedsortedfin = zasummedsorted.reset_index().drop(["index"], axis=1)

    zasummedsortedfin.to_csv(filepath,index=False)
    
    return zasummedsortedfin

In [27]:
relevantcols = ['main_artist','streams', "source"]
grouplist = ['source']
sumcol = 'streams'
sortcol = 'streams'
isascending = False
isYear = False
filepath = "Classification/sortedbysourceonlyZA2022.csv"
zasummedsortedstreamssource = groupbysum(all_files_za, 2022, relevantcols, grouplist, sumcol, sortcol, isascending, isYear, filepath)

In [28]:
zasummedsortedfin

,source,streams
0,Columbia,94537287
1,Republic Records,64181333
2,Universal Music (Pty) Ltd.,50204548
3,Atlantic Records UK,40200144
4,OVO,38588701
...,...,...
717,Southeastern Records,2236
718,DrumbossMusik,2131
719,Defected Records,2090
720,Coke Studio Africa,2086


In [29]:
def groupbyvalue(za2022, idcol, relevantcols, groupcol, countdistinctcol):
    za2022df = za2022[relevantcols].groupby([groupcol]).agg({countdistinctcol: pd.Series.nunique})
    new2022df = pd.DataFrame(za2022df)
    new2022df.columns = ["count_" + countdistinctcol]
    
    new2022df = new2022df.reset_index()
    
    new2022dfunique = new2022df[[idcol, "count_" + countdistinctcol]]
    
    new2022dfunique = new2022dfunique.drop_duplicates()
    
    return new2022dfunique

def sortvaluesbyorder(new2022dfunique, countdistinctcolname, isascending):
    zacountsorted = new2022dfunique.sort_values(by=countdistinctcolname, ascending=isascending)
    zacountsorted = zacountsorted.reset_index()
    zacountsorted.drop(["index"], inplace=True, axis=1)
    zacountsorted = zacountsorted.dropna()
    
    return zacountsorted

In [34]:
relevantcols = ['main_artist', 'track_name', 'streams', "source"]
groupcol = "source"
countdistinctcol = "main_artist"

idcol = "source"
new2022dfunique = groupbyvalue(all_files_za, idcol, relevantcols, groupcol, countdistinctcol)

print(list(new2022dfunique.columns), new2022dfunique.shape)

countdistinctcolname = "count_" + countdistinctcol

zacountsortedartistspersource = sortvaluesbyorder(new2022dfunique, countdistinctcolname, False)

# zacountsorted.to_csv("Classification/sortedsourcesartists_ZA2022.csv", index=False)

['source', 'count_main_artist'] (722, 2)
['source', 'count_track_name'] (722, 2)


In [35]:
relevantcols = ['main_artist', 'track_name', 'streams', "source"]
groupcol = "source"
countdistinctcol = "track_name"

idcol = "source"
new2022dfunique = groupbyvalue(all_files_za, idcol, relevantcols, groupcol, countdistinctcol)

print(list(new2022dfunique.columns), new2022dfunique.shape)

countdistinctcolname = "count_" + countdistinctcol

zacountsortedtrackspersource = sortvaluesbyorder(new2022dfunique, countdistinctcolname, False)


['source', 'count_track_name'] (722, 2)


In [37]:
zasummedsortedfin.shape, zacountsortedartistspersource.shape, zacountsortedtrackspersource.shape

((722, 2), (722, 2), (722, 2))

In [38]:
zasourcedf1 = zasummedsortedfin.merge(zacountsortedartistspersource, left_on=["source"], right_on=["source"], how="inner")

In [40]:
zasourcedf = zasourcedf1.merge(zacountsortedtrackspersource, left_on=["source"], right_on=["source"], how="inner")

In [41]:
zasourcedf

,source,streams,count_main_artist,count_track_name
0,Columbia,94537287,54,200
1,Republic Records,64181333,25,178
2,Universal Music (Pty) Ltd.,50204548,45,192
3,Atlantic Records UK,40200144,14,65
4,OVO,38588701,2,65
...,...,...,...,...
717,Southeastern Records,2236,1,1
718,DrumbossMusik,2131,1,1
719,Defected Records,2090,1,1
720,Coke Studio Africa,2086,1,1


In [42]:
zasourcedf.to_csv("Classification/sourcedetailsall.csv", index=False)

##### Find South African artists

In [43]:
# Read the south african file
files_za_artists = pd.read_csv("Classification/JustSouthAfricansOnZAChart.csv")

In [44]:
files_za_artists.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'Lower Artist'],
      dtype='object')

In [50]:
zasources = files_za_artists[["source"]].drop_duplicates()

In [53]:
justzasourcedf = zasourcedf.merge(zasources, left_on=["source"], right_on=["source"], how="right")

In [54]:
justzasourcedf.to_csv("Classification/ZA_sourcedetails.csv", index=False)